In [1]:
import numpy as np 
import pandas as pd 

In [2]:
import re
import string
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split

In [4]:
df_reddit = pd.read_csv(r"C:\Users\Anupam\OneDrive\Desktop\nlp_mini_project\Reddit_dataset\Reddit_Data.csv")

In [5]:
df_reddit.head()

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [6]:
df_reddit.rename(columns={"clean_comment": "comment","category":"label"}, inplace = True)

In [7]:
df_reddit = df_reddit.dropna()

In [8]:
punct = string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [9]:
stopWords = stopwords.words('english')

In [10]:
ps = nltk.PorterStemmer()
wn = nltk.WordNetLemmatizer()

In [11]:
def cleanData(text):
    text = text.lower()
    text = re.sub(r"[^A-Za-z0-9]",' ', text)
    text = ''.join([char for char in text if char not in punct])
    text = [wn.lemmatize(word) for word in text.split(' ') if ((word not in stopWords) & len(word)!=0)]
    return ' '.join(text)

In [12]:
df_reddit['comment'] = df_reddit['comment'].apply(cleanData)

In [13]:
df_reddit.head()

,comment,label
0,never tried explain still stare puzzled strang...,1
1,lot considering sin suffering thing suffering ...,1
2,seriously say thing first get complex explain ...,-1
3,learned teach different focus paper without,0
4,benefit may thich might discussion loved paral...,1


In [14]:
df_reddit = df_reddit[df_reddit.label != 0]
df_reddit.label.value_counts()

 1    15830
-1     8277
Name: label, dtype: int64

In [15]:
df_reddit.head()

,comment,label
0,never tried explain still stare puzzled strang...,1
1,lot considering sin suffering thing suffering ...,1
2,seriously say thing first get complex explain ...,-1
4,benefit may thich might discussion loved paral...,1
5,sit watch episode becomes episode faith discus,-1


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(max_features=1000, binary=True)
X = vect.fit_transform(df_reddit.comment)

X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [17]:
X = df_reddit.comment
y = df_reddit.label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.count())
print(X_test.count())


19285
4822


In [18]:
vect = CountVectorizer(max_features=1000, binary=True)

X_train_vect = vect.fit_transform(X_train)

In [19]:
counts = df_reddit.label.value_counts()
print(counts)

print("\nPredicting only -1 = {:.2f}% accuracy".format(counts[-1] / sum(counts) * 100))

 1    15830
-1     8277
Name: label, dtype: int64

Predicting only -1 = 34.33% accuracy


In [20]:
from imblearn.over_sampling import SMOTE

sm = SMOTE()

X_train_res, y_train_res = sm.fit_resample(X_train_vect, y_train)

In [21]:
unique, counts = np.unique(y_train_res, return_counts=True)
print(list(zip(unique, counts)))

[(-1, 12656), (1, 12656)]


In [22]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()

nb.fit(X_train_res, y_train_res)

nb.score(X_train_res, y_train_res)

0.738108407079646

In [23]:
X_test_vect = vect.transform(X_test)

y_pred = nb.predict(X_test_vect)

y_pred

array([ 1, -1, -1, ..., -1,  1, -1], dtype=int64)

In [24]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

print("Accuracy: {:.2f}%".format(accuracy_score(y_test, y_pred) * 100))
print("\nF1 Score: {:.2f}".format(f1_score(y_test, y_pred) * 100))
print("\nCOnfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 68.19%

F1 Score: 73.64

COnfusion Matrix:
 [[1145  503]
 [1031 2143]]


In [25]:
predict = nb.predict(vect.transform(["Good job"]))
if predict == 1:
    print("The sentiment is positive")
else:
    print("The sentiment is negative")

The sentiment is positive
